In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leinad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle
from tensorflow import keras

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
with open("intents.json") as file:
	data = json.load(file)

In [4]:
random.shuffle(data["intents"])

# Split data into training and test sets
split_ratio = 0.8  # 80% for training, 20% for testing
split_index = int(len(data["intents"]) * split_ratio)
train_data = data["intents"][:split_index]
test_data = data["intents"][split_index:]

In [5]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [6]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


INFO:tensorflow:Restoring parameters from c:\Users\Leinad\Documents\Tech_Villain\Leinadpython\firstaid\model.tflearn


In [7]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [8]:
def chat():
	print("Start Talking with the bot(type quit to stop!")
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']	
			print(random.choice(responses))
			print("\n")
		else:
			print("I didnt get that, try again")


In [9]:
chat()

Start Talking with the bot(type quit to stop!
I didnt get that, try again
I didnt get that, try again


In [139]:
import numpy as np
from sklearn.metrics import recall_score, f1_score

In [140]:
y_true = []  # Ground truth labels
y_pred = []  # Predicted labels

for intent in test_data:
    for pattern in intent["patterns"]:
        test_input = bag_of_words(pattern, words)
        prediction = model.predict([test_input])
        predicted_label = labels[np.argmax(prediction)]
        y_pred.append(predicted_label)
        # Assuming you have access to the ground truth label for each test sample
        y_true.append(intent["tag"])

# Calculate recall and F1 score
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)
f1score = f1_score(y_true, y_pred, average='weighted')

print("Recall: ", recall)
print("F1 Score: ", f1score)

Recall:  0.02564102564102564
F1 Score:  0.015779092702169623


array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [110]:
from sklearn.model_selection import train_test_split
train_data, eval_data, train_labels, eval_labels = train_test_split(training, output, test_size=0.2, random_state=42)

In [114]:
from sklearn.metrics import accuracy_score
predictions = model.predict(eval_data)

# Convert the predictions to class labels (index with the highest probability)
predicted_labels = [labels[prediction.argmax()] for prediction in predictions]

# Calculate accuracy as the evaluation metric
accuracy = accuracy_score([labels[label.argmax()] for label in eval_labels], predicted_labels)
print("Accuracy:", accuracy)

Accuracy: 0.02564102564102564


In [116]:
import numpy as np
def calculate_tp_fp_fn(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true == 1, y_pred == 1))
    fp = np.sum(np.logical_and(y_true == 0, y_pred == 1))
    fn = np.sum(np.logical_and(y_true == 1, y_pred == 0))
    return tp, fp, fn

# Evaluate precision and recall
def evaluate_precision_recall(y_true, y_pred):
    tp, fp, fn = calculate_tp_fp_fn(y_true, y_pred)
    precision = tp / (tp + fp + 1e-8)  # Add small epsilon to avoid division by zero
    recall = tp / (tp + fn + 1e-8)     # Add small epsilon to avoid division by zero
    return precision, recall

# Assuming you already have the predictions and ground truth labels
# predictions contains the predicted labels (converted to class indices)
# eval_labels contains the ground truth labels (converted to class indices)

# Convert predictions and eval_labels to binary format for multilabel evaluation
y_pred_binary = np.zeros_like(predictions)
y_pred_binary[np.arange(len(predictions)), predictions.argmax(axis=1)] = 1

eval_labels_binary = np.zeros_like(eval_labels)
eval_labels_binary[np.arange(len(eval_labels)), eval_labels.argmax(axis=1)] = 1

# Evaluate precision and recall
precision, recall = evaluate_precision_recall(eval_labels_binary, y_pred_binary)

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.025641025634451017
Recall: 0.025641025634451017


In [2]:
abs(12 // 5)

2